## Импортируйте необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk

import re

import gc

In [ ]:
df_review = pd.read_csv('IMDB Dataset.csv')

## Проанализируйте выборку (сбалансированность классов, пропуски и пр.)

In [ ]:
review = df_review
review

In [ ]:
review.sentiment.value_counts()

In [ ]:
review.info()

## Примените LabelEncoding целевой переменной (sklearn)

In [ ]:
le = LabelEncoder()
df_review['sentiment_encoded'] = le.fit_transform(df_review['sentiment'])
df_review

## Препроцессинг текста (надо применить к каждому отзыву):

приведите все символы к нижнему регистру, 
удалить знаки препинания, 
цифры

In [ ]:
def clean(text):
    text = text.lower()
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'<[^>]+>',' ', text)
    text = re.sub(r'[^\w\s]',' ', text)
#     text = re.sub(r'@\w+',' ',text)
#     text = re.sub(r'#\w+', ' ', text)
#     text = re.sub(r'<.*?>',' ', text)
#     punctuation = ['']
#     for i in punctuation:
#         text = text.replace(i, '')
#     text = text.replace("'", ' ')
#     text = text.replace('.', " ")
#     text = text.replace('/', " ")
#     text = text.replace('?', " ")
#     text = text.replace(':', " ")
#     text = text.replace('"', " ")
#     text = text.replace(',', " ")
#     text = text.replace(':', "")
    return text

In [ ]:
text = df_review['review']
text.head()

In [ ]:
text = text.apply(clean)
text.head(15)

провести лемматизацию, 
очистить от стоп-слов (сравните stopwords в библиотеке nltk и gensim, 
провести токенизацию

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
wn_lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatized_text = []
for review in text:
    lemmatized_text.append(' '.join([wn_lemmatizer.lemmatize(word) for word in review.split()]))

In [ ]:
reg_tokenizer = RegexpTokenizer('\w+')

In [ ]:
tokenized_text = reg_tokenizer.tokenize_sents(lemmatized_text)

In [ ]:
sw = stopwords.words()

In [ ]:
clean_tokenized_review = [] 
for i, element in enumerate(tokenized_text):
    if i % 2000 == 0: print(i, end = ' ')
    clean_tokenized_review.append(' '.join([word for word in element if word not in sw]))

In [ ]:
df = pd.concat([pd.Series(clean_tokenized_review, name='review'), 
                pd.Series(df_review['sentiment_encoded'], name='sentiment')], 
               axis=1)

In [ ]:
df

In [ ]:
df.to_csv('clean_reviews', index=False)

In [2]:
df = pd.read_csv('clean_reviews')

In [4]:
cvec = CountVectorizer(ngram_range=(1, 2))
tfid = TfidfVectorizer(ngram_range=(1, 2))

In [5]:
cvec_representation = cvec.fit_transform(df['review'])
tfid_representation = tfid.fit_transform(df['review'])

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(cvec_representation,
                                                    df['sentiment'], test_size=0.2, random_state=900)

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.model_selection import cross_val_score

In [ ]:
# mnb = MultinomialNB()
# lrc = LogisticRegression()
# svc = SVC()

In [ ]:
# clfs = {
#     'MultiNB' : mnb, 
#     'LogisticRegression' : lrc,
#     'SVM' : svc
# }

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [11]:
rf = RandomForestClassifier()

In [12]:
rf.fit(x_train, y_train)
# y_pred_rf = rf.predict(x_test)
# print(f'RandomForest accuracy: {accuracy_score(y_test, y_pred_rf)}')

RandomForestClassifier()

In [13]:
y_pred_rf = rf.predict(x_test)
print(f'RandomForest accuracy: {accuracy_score(y_test, y_pred_rf)}')

RandomForest accuracy: 0.8593
